In [ ]:
import random
import time
import pickle

import heapq
from copy import deepcopy
from collections import deque, defaultdict
from termcolor import colored


class Simulation:
	"""Sets up a simulation and runs it providing a score for a particular set of schedules"""

	# Constructor
	def __init__(self, D, I, S, V, F):
		"""
		Constructor for class Simulation

		:type D: int
		:param D: Duration of simulation
		:type I: int
		:param I: Number of intersections
		:type S: int
		:param S: Number of streets
		:type V: int
		:param V: Number of cars
		:type F: int
		:param F: Bonus points awarded when each car finishes it's journey
		"""

		self._D = D
		"""Duration of simulation"""
		self._I = I
		"""Number of intersections"""
		self._S = S
		"""Number of streets"""
		self._V = V
		"""Number of cars"""
		self._F = F
		"""Bonus points awarded when each car finishes it's journey"""

		self._street_ids = {}
		"""
		{street_name: street_id, ...}
		"""
		self._streets = [tuple() for _ in range(self._S)]
		"""
		[time_taken_to_cross_street, street_name, start_intersection, end_itersection] at index street_id
		"""

		self._cars = []
		"""
		Tuple of current position of car (as an index) and path followed by each car in the form of a list of street IDs at index car_id
		"""

		self._intersections = []
		"""
		Tuple of {ids_of_incoming_streets...} and {ids_of_outgoing_streets...} for each intersection at index intersection_id
		"""

		self._setup_complete = False
		"""To store status of setup of simulation"""

		self._timetable = []
		"""To store the set of green signals in a single signal cycle of each intersection"""

		self._timetable_initialized = False
		"""To store status of initialization of signal timetable"""

		self._street_queues = []
		"""To store queues of cars and the time at which they will reach the end of the street for each street, indexed by street_id"""

		self._t = 0
		"""To store the current time (in second) of the simulation"""

		self._score = 0
		"""To store the score which has been achieved until now i.e. till time self._t"""

		self._simulation_complete = False
		"""To store the status of the simulation"""

		self._schedules = []
		"""To store the schedules provided to create the timetable (for viewing purposes only)"""

		self._final_countdown = []
		"""To store the cars which are on their final road"""

		self._number_of_completed_journeys = 0
		"""To store how many cars have completed their journey"""

		self._history = {'streets': defaultdict(lambda: defaultdict(lambda: [-1, -1])), 'cars': defaultdict(lambda: defaultdict(lambda: [-1, -1]))}
		"""To store the history of the entire simulation"""


	# Methods to set up the simulation
	def add_street(self, street) -> bool:
		"""
		To be used during setup. Adds a street to the simulation.
		:type street: tuple[int, str, int, int]
		:param street: (time_taken_to_cross_street, street_name, start_intersection, end_intersection)
		:return: Street successfully added or not
		"""

		# Check if setup is already complete
		if self._setup_complete:
			return False

		# Check if street already exists
		street_name = street[2]
		if street_name in self._street_ids:
			return False

		# Add street to simulation
		street_id = len(self._street_ids)
		self._street_ids[street_name] = street_id
		self._streets[street_id] = (street[3], street_name, street[0], street[1])

		return True

	def add_car(self, car) -> bool:
		"""
		To be used during setup. Adds a car to the simulation.
		:type car: tuple
		:param car: (number_of_streets, [list_of_street_names...])
		:return: Car successfully added or not
		"""

		# Check if setup is already complete
		if self._setup_complete:
			return False

		# Check if the given input is of valid format
		if len(car[1]) != car[0]:
			return False

		# Add car to simulation
		self._cars.append([0, self.get_street_ids(car[1])])

		return True

	def finish_setup(self) -> tuple:
		"""
		Method to tell the simulation the setup is complete.
		Identifies all intersections and preps the data for simulation
		:return: Setup has been successfully completed or not
		"""

		# Check if all the streets have been added to the simulation
		if len(self._streets) != self._S:
			return False, []

		# Check if all the cars have been added to the simulation
		if len(self._cars) != self._V:
			return False, []

		# Identify all the intersections
		self._identify_intersections()

		# Check if the number of intersections are correct
		if len(self._intersections) != self._I:
			return False, []

		# Flag setup as complete
		self._setup_complete = True

		# Initialize variables to use during simulation
		self._timetable = []
		self._setup_street_queues()

		return True, list(self._street_ids.keys())


	# Methods to perform the simulation
	def tick(self, allow_print=True, perform_pre_checks=True) -> bool:
		"""
		Runs the simulation for a single second and updates all the corresponding values

		:param allow_print: Whether or not to print changes in the simulation
		:param perform_pre_checks:
		:return: whether or not the tick was successful
		"""

		if perform_pre_checks and (not self._setup_complete or not self._timetable_initialized):
			return False

		if perform_pre_checks and self._simulation_complete:
			if allow_print:
				print(f'Simulation already complete! Score = {self._score}')
			return False

		if allow_print:
			print(f'\nTick {self._t}:')

		# Check if there are cars ready to finish
		while self._final_countdown:
			earliest_completion = heapq.nsmallest(1, self._final_countdown)

			if earliest_completion[0][0] <= self._t:
				car_id = earliest_completion[0][1]

				self._score += self._F + self._D - self._t
				heapq.heappop(self._final_countdown)

				self._number_of_completed_journeys += 1
			else:
				break

		if self._number_of_completed_journeys != self._V:
			for signal_cycle in self._timetable:
				if not signal_cycle:
					continue

				street_id = signal_cycle[self._t % len(signal_cycle)]

				queue = self._street_queues[street_id]

				if queue and queue[0][1] <= self._t:
					car_id = queue[0][0]
					tick_of_arrival = queue[0][1]
					queue.popleft()

					car = self._cars[car_id]
					car[0] += 1

					intersection_id = self._streets[street_id][3]

					new_street_id = car[1][car[0]]
					new_queue = self._street_queues[new_street_id]

					self._history['cars'][car_id][street_id][1] = self._t
					self._history['cars'][car_id][new_street_id][0] = self._t
					self._history['streets'][street_id][car_id][1] = self._t
					self._history['streets'][new_street_id][car_id][0] = self._t

					# self._history['cars'][car_id][0].append(self._t - tick_of_arrival)
					# self._history['cars'][car_id][1].append((street_id, self._t))
					# self._history['streets'][street_id][0].append((1, car_id, self._t))
					# self._history['streets'][street_id][1].append(self._t - tick_of_arrival)
					# self._history['streets'][new_street_id][0].append((0, car_id, self._t))
					# self._history['intersections'][intersection_id][car_id] = (tick_of_arrival, street_id, self._t)

					time_to_cross = self._streets[new_street_id][0]

					if car[0] + 1 == len(car[1]):
						heapq.heappush(self._final_countdown, (self._t + time_to_cross, car_id))
					else:
						new_queue.append((car_id, self._t + time_to_cross))

					if allow_print:
						print(f'Car {car_id} passed a signal:\t\t"{self._streets[street_id][1]}"  ->  "{self._streets[new_street_id][1]}"')

		self._t += 1

		if self._t == self._D:
			self._simulation_complete = True

	def simulate(self, till=None, allow_print=True) -> bool:
		"""
		Performs the entire simulation from the current time self._t to the end of the simulation self._D

		:param till: Specify tick until which simulation should run
		:param allow_print: Whether or not to print scores at every 500 seconds
		:return: whether or not the simulation was successful
		"""

		if not self._setup_complete or not self._timetable_initialized:
			return False

		if self._simulation_complete:
			if allow_print:
				print(f'Simulation already complete! Score = {self._score}')
			return False

		if till is None or till > self._D:
			till = self._D

		if allow_print:
			print(f'Starting simulation at time = {self._t}')

		for _ in range(till - self._t):
			self.tick(False, False)

			if allow_print and self._t % 500 == 0:
				print(f'Time: {self._t}\t\tScore: {self._score}')

		if allow_print:
			print(f'\nSimulation complete!\nTime: {self._t}s\t\tScore: {self._score}')
			print(f'{self._number_of_completed_journeys} cars completed their journey')

		return True

	def initialize_timetable(self, schedules, reset=True, empty_schedule=False) -> bool:
		"""
		Creates a list of street ids for each intersection of length equal to the cycle length

		:param reset: Whether or not to reset the simulation
		:type schedules: list[list[tuple]]
		:param schedules: Schedule of all the intersections. list of (street_id, duration_to_remain_green)... for each interection at index intersection_id
		:param empty_schedule: To inform simulation that the provided schedules are empty and are for dummy purposes only
		:return: Timetable intialized successfully or not
		"""

		# Check if setup has been completed or not
		if not self._setup_complete: return False

		# Reset simulation in case it had been started before
		if reset:
			self.reset()

		self._schedules = deepcopy(schedules)

		# Initialize the timetable list
		self._timetable = [[] for _ in range(len(schedules))]

		if not empty_schedule:
			for intersection_id, schedule in enumerate(schedules):
				self.update_timetable(intersection_id, schedule, False, False)

		# Set the status as timetable initialized
		self._timetable_initialized = True

		return True

	def update_timetable(self, intersection_id, schedule, reset=True, update_cached_schedule=True) -> bool:
		"""
		Updates the existing timetable for a particular intersection with the given schedule

		:param update_cached_schedule: Whether or not to update self._schedules with the given schedule
		:param intersection_id: ID of intersection who's schedule has changed
		:param schedule: New schedule to be incorporated
		:param reset: whether or not to reset the simulation back to time 0
		:return: Timetable updated successfully or not
		"""

		# Check if timetable has been initialized already
		if not self._timetable_initialized:
			return False

		# Save the signal cycle length of that intersection
		self._timetable[intersection_id] = []

		# Add the ticks at which each signal changes and the corresponding streets

		for street_id, duration in schedule:
			self.timetable[intersection_id].extend([street_id] * duration)

		if update_cached_schedule:
			self._schedules[intersection_id] = schedule

		if reset:
			self.reset()

		return True

	def reset(self) -> bool:
		"""
		Reset simulation back to time = 0
		:return: Reset successful or not
		"""

		self._score = 0
		self._t = 0
		self._simulation_complete = False
		self._number_of_completed_journeys = 0
		self._history = {'streets': defaultdict(lambda: defaultdict(lambda: [-1, -1])), 'cars': defaultdict(lambda: defaultdict(lambda: [-1, -1]))}

		for car in self.cars:
			car[0] = 0
		self._setup_street_queues()
		self.initialize_timetable(self._schedules, False)

		return True


	# Helper methods
	def print_car_details(self, car_id) -> bool:
		if 0 > car_id >= self._V:
			return False

		if not self._timetable_initialized:
			return False

		car_history = self._history['cars'][car_id]

		print(f'CAR ID:\t{car_id}')
		print(f'ACTIVITY:\n\t', end='')

		if self._cars[car_id][0] == 0:
			print(colored(f'Car has not yet begun it\'s journey', 'red'))
		else:
			path = self._cars[car_id][1]
			print(colored(f'Left street {path[0]} at t={car_history[path[0]][1]}'.ljust(30), 'green'), end=' -> ')

			i = 1
			while i < len(path):
				travel_time = self._streets[path[i]][0]
				if car_history[path[i]][1] == -1:
					if self._t < car_history[path[i]][0] + travel_time:
						print(colored(f'Still in journey', 'cyan'))
					else:
						print(colored(f'Car waiting at an intersection', 'cyan'))

					break
				else:
					print(colored(f'Travelling for {travel_time}s'.ljust(25), 'yellow'), end=' -> ')
					print(colored(f'Waited for {car_history[path[i]][1] - car_history[path[i]][0] - travel_time}s'.ljust(20), 'red'), end=' -> \n\t')
					print(colored(f'Left street {path[i]} at t={car_history[path[i]][1]}'.ljust(30), 'green'), end=' -> ')

				i += 1

			if i == len(path):
				print(colored(f'Journey completed at t={car_history[path[-1]][0] + self._streets[path[-1]][0]}', 'cyan'))

		return True

	def print_street_details(self, street_id) -> bool:
		if 0 > street_id >= self._S:
			return False

		if not self._timetable_initialized:
			return False

		street_history = self._history['streets'][street_id]
		street_time_to_cross = self._streets[street_id][0]

		print(f'STREET ID:\t{street_id}')
		print(f'TRAVEL TIME:\t{street_time_to_cross}')
		print(f'TOTAL TIME CARS WAITED: {sum([activity[1] - activity[0] - street_time_to_cross for _, activity in street_history.items() if activity[1] != -1])}')
		print(f'ACTIVITY:')

		heading_printed = False
		for car_id, activity in street_history.items():
			if activity[1] == -1:
				if not heading_printed:
					print(f'CARS STILL ON THE STREET:')
					heading_printed = True

				if self._t < activity[0] + street_time_to_cross:
					print(colored(f'\tCar {car_id}: travelling', 'yellow'))
				else:
					print(colored(f'\tCar {car_id}: waiting', 'red'))

			else:
				print(colored(f'\tCar {car_id} arrived at t={activity[0]}'.ljust(30), 'yellow'), end='')
				print(colored(f'Waited for {activity[1] - activity[0] - street_time_to_cross}s'.ljust(20), 'red'), end='')
				print(colored(f'Left at t={activity[1]}'.ljust(30), 'green'))

		return True

	def print_intersection_details(self, intersection_id):
		if 0 > intersection_id >= self._I:
			return False

		if not self._timetable_initialized:
			return False

		print(f'INTERSECTION ID:\t{intersection_id}')
		print(f'INCOMING STREETS QUEUES (n={len(self._intersections[intersection_id][0])}):')

		empty_count = 0
		for street_id in sorted(self._intersections[intersection_id][0]):
			if self._street_queues[street_id]:
				print(f'\tStreet {street_id}: ', end='')
				for car in self._street_queues[street_id]:
					if car[1] > self._t:
						print(colored(car[0], 'yellow'), end = ' ')
					else:
						print(colored(f'{car[0]} ({self._t - car[1]}s)', 'red'), end=' ')
				print()
			else:
				empty_count += 1

		print(f'\t({empty_count} street queues empty)')

		# print(f'OUTGOING STREETS (n={len(self._intersections[intersection_id][1])}):')
		# print('\t', self._intersections[intersection_id][1])

		print(f'PAST:')
		# TODO: ADD

	def get_copy(self) -> 'Simulation':
		copy = Simulation(self._D, self._I, self._S, self._V, self._F)
		copy._street_ids = self._street_ids
		copy._streets = self._streets
		copy._cars = self._cars
		copy._intersections = self._intersections
		copy.finish_setup()

		return copy

	def get_street_ids(self, street_names) -> list:
		"""
		Returns a list of street IDs corresponding to the street names provided
		:param street_names: List of street names who's IDs are required
		:return: List of required street IDs in their corresponding indexes
		"""

		return [self.street_ids[street_name] for street_name in street_names]

	def _identify_intersections(self) -> None:
		"""Identifies all the intersections in the simulation and stores it in self._intersections"""
		self._intersections = [(set(), set()) for _ in range(self._I)]

		for street_id, street_details in enumerate(self._streets):
			self._intersections[street_details[2]][1].add(street_id)
			self._intersections[street_details[3]][0].add(street_id)

	def _setup_street_queues(self) -> None:
		"""
		Sets up all the queues for each street in self._street_queues
		:return:
		"""

		self._street_queues = [deque() for _ in range(self._S)]

		for i, car in enumerate(self._cars):
			self._street_queues[car[1][0]].append((i, 0))

	def _find_score(self):
		pass

	# Property methods
	@property
	def streets(self):
		return self._streets

	@property
	def street_ids(self):
		return self._street_ids

	@property
	def cars(self):
		return self._cars

	@property
	def intersections(self):
		return self._intersections

	@property
	def timetable(self):
		return self._timetable

	@property
	def street_queues(self):
		return self._street_queues

	@property
	def t(self):
		return self._t

	@property
	def score(self):
		return self._score

	@property
	def number_of_completed_journeys(self):
		return self._number_of_completed_journeys

	@property
	def history(self):
		return self._history








# ----- Defining Global Variables -----
input_path = r'../input/hashcode-2021-oqr-extension/hashcode.in'
output_path = r'./submission.csv'


# ----- Reading input file & setting up simulation -----
with open(input_path) as file:
	D, I, S, V, F = map(int, file.readline().split())
	print(f'Duration: {D}\nNumber of Intersections: {I}\nNumber of Streets: {S}\nNumber of Cars: {V}\nBonus points: {F}\n')

	simulation = Simulation(D, I, S, V, F)

	for s in range(S):
		line = file.readline().split()
		street = (int(line[0]), int(line[1]), line[2], int(line[3]))

		if not simulation.add_street(street):
			raise Exception

	for v in range(V):
		line = file.readline().split()
		car = (int(line[0]), line[1:])

		if not simulation.add_car(car):
			raise Exception

	setup_completed, streets_at_ids = simulation.finish_setup()
	if setup_completed:
		print('Setup complete')
	else:
		print('Could not set up simulation')

	del setup_completed


# ----- Find those streets which don't have any traffic to exclude from optimization ------
street_ids_without_traffic = {*range(S)}
for car in simulation.cars:
	for street_id in car[1]:
		street_ids_without_traffic.discard(street_id)

# print(street_ids_without_traffic)	# len = 9779


# ----- Create set of initial schedules by being greedy -----
# print(f'Creating a set of greedy schedules')
# start_time = time.time()

# dummy = simulation.get_copy()

# cycle_lengths_and_occupied = [(len(intersection[0] - street_ids_without_traffic), set()) for intersection in dummy.intersections]
# schedules = [[(None, 1) for _ in range(cycle_lengths_and_occupied[intersection_id][0])] for intersection_id in range(I)]
# street_ids_included_in_schedule = set()
# street_queues = deepcopy(dummy.street_queues)
# cars = deepcopy(dummy.cars)
# streets = deepcopy(dummy.streets)

# for t in range(D):
# 	for street_id in set(range(S)) - street_ids_without_traffic:
# 		if street_queues[street_id] and street_queues[street_id][0][1] <= t:
# 			intersection_id = streets[street_id][3]
# 			schedule = schedules[intersection_id]
# 			cycle_length_and_occupied = cycle_lengths_and_occupied[intersection_id]

# 			index = int(t % cycle_length_and_occupied[0])

# 			if street_id not in street_ids_included_in_schedule:
# 				street_ids_included_in_schedule.add(street_id)

# 				while index in cycle_length_and_occupied[1]:
# 					index = (index + 1) % cycle_length_and_occupied[0]

# 				cycle_length_and_occupied[1].add(index)

# 				schedule[index] = (street_id, 1)

# 			if schedule[index][0] == street_id:
# 				car_id = street_queues[street_id][0][0]
# 				street_queues[street_id].popleft()

# 				car = cars[car_id]
# 				car[0] += 1

# 				if car[0] + 1 != len(car[1]):
# 					new_street_id = car[1][car[0]]
# 					street_queues[new_street_id].append((car_id, t + streets[new_street_id][0]))



# 	if t % 500 == 0:
# 		print(f'Time = {t}\t\tNumber of streets allotted a signal = {len(street_ids_included_in_schedule)}')

# for i in range(len(schedules)):
# 	schedules[i] = [signal for signal in schedules[i] if signal[0] != None]

# print(f'\nTime taken to create initial schedules = {round(time.time() - start_time, 2)}s')

# del dummy
# del cycle_lengths_and_occupied
# del street_ids_included_in_schedule
# del street_queues
# del cars
# del streets


# with open('../input/hashcode-21-traffic-signalling/initial_schedules.out', 'rb') as schedule_file:
# 	schedules = pickle.load(schedule_file)


# # ----- Provide schedules to the simulation -----
# start_time = time.time()
# simulation.initialize_timetable(schedules)
# print(f'Time taken to create timetable = {round(time.time() - start_time, 2)}s')


# # ----- Simulate -----
# print(f'\n\nSimulation with greedy schedules')
# start_time = time.time()
# simulation.simulate()
# print(f'\nTime taken to complete simulation = {round(time.time() - start_time, 2)}s')

In [ ]:
def get_cost(arrivals, index, cycle_length):
	index = index % cycle_length
	t = index
	cost = 0

	for i in range(len(arrivals)):
		if t < arrivals[i]:
			t = (arrivals[i] // cycle_length) * cycle_length + index
			if t < arrivals[i]: t += cycle_length

		cost += t - arrivals[i]
		t += cycle_length

	return cost


def get_cost_of_intersection(intersection_id):
	cost = 0
	street_ids = simulation.intersections[intersection_id][0]

	for street_id in street_ids:
		ttc = simulation.streets[street_id][0]
		cost += sum([activity[1] - activity[0] for _, activity in simulation.history['streets'][street_id].items() if
					 activity[1] != -1])

	return cost


# for i in range(8):

# 	intersection_id = None
# 	max_cost = 0

# 	for i in range(I):
# 		cost = get_cost_of_intersection(i)
# 		if cost > max_cost:
# 			max_cost = cost
# 			intersection_id = i

# 	street_ids = simulation.intersections[intersection_id][0] - street_ids_without_traffic
# 	arriving_ticks = defaultdict(list)

# 	for street_id in street_ids:
# 		for car_id, activity in simulation.history['streets'][street_id].items():
# 			if activity[1] == -1:
# 				break

# 			arriving_ticks[street_id].append(activity[0] + simulation.streets[street_id][0])

# 	arriving_ticks = list(sorted(arriving_ticks.items(), key=lambda x: sum(x[1]), reverse=True))
# 	used_indexes = set()
# 	schedule = schedules[intersection_id]

# 	for street_id, ticks in arriving_ticks:

# 		best_index = 0
# 		best_cost = 1000000

# 		for x in range(0, len(street_ids)):
# 			if x in used_indexes: continue

# 			cost = get_cost(ticks, x, len(street_ids))

# 			if cost < best_cost:
# 				best_index = x
# 				best_cost = cost

# 		schedule[best_index] = (street_id, 1)
# 		used_indexes.add(best_index)

# 	simulation.update_timetable(intersection_id, schedule)
# 	simulation.simulate(allow_print=True)

with open(r'../input/hashcode21-traffic-signalling/good_answer.in', 'rb') as file:
    schedules = pickle.load(file)

    
submission = (schedules, streets_at_ids)

# ----- Save schedule to output file -----

# Checking if everything is in order
assert len(submission[0]) == I, 'Please create schedules for all the intersections'


# # Saving to output file
with open(output_path, 'w') as out_file:
	number_of_active_intersections = 0
	for schedule in submission[0]:
		if len(schedule) != 0:
			number_of_active_intersections += 1

	out_file.write(str(number_of_active_intersections) + '\n')

	for intersection_id in range(I):
		if len(submission[0][intersection_id]) == 0:
			continue

		out_file.write(str(intersection_id) + '\n')

		E = len(submission[0][intersection_id])
		out_file.write(str(E) + '\n')

		for j in range(E):
			out_file.write(str(submission[1][submission[0][intersection_id][j][0]]))
			out_file.write(' ')
			out_file.write(str(submission[0][intersection_id][j][1]))
			out_file.write('\n')